<a href="https://colab.research.google.com/github/DeenReinierCrusem/CPEN65-1/blob/main/Laboratory_Exercise_2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Laboratory Exercise 2: Classification


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors':
[3, 4, 5]}]
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)
GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
 param_grid=[{'n_neighbors': [3, 4, 5],
 'weights': ['uniform', 'distance']}],
 verbose=3)
grid_search.best_params_
{'n_neighbors': 4, 'weights': 'distance'}
grid_search.best_score_
0.9716166666666666
from sklearn.metrics import accuracy_score
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from scipy.ndimage.interpolation import shift
def shift_image(image, dx, dy):
 image = image.reshape((28, 28))
 shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
 return shifted_image.reshape([-1])
image = X_train[1000]
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)
plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest",
cmap="Greys")
plt.subplot(132)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28),
interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28),
interpolation="nearest", cmap="Greys")
plt.show()

X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
 for image, label in zip(X_train, y_train):
 X_train_augmented.append(shift_image(image, dx, dy))
 y_train_augmented.append(label)
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)
KNeighborsClassifier(n_neighbors=4, weights='distance')
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
import os
import urllib.request
TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handsonml2/master/datasets/titanic/"
def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
if not os.path.isdir(path):
 os.makedirs(path)
for filename in ("train.csv", "test.csv"):
 filepath = os.path.join(path, filename)
 if not os.path.isfile(filepath):
 print("Downloading", filename)
 urllib.request.urlretrieve(url + filename, filepath)
fetch_titanic_data()
import pandas as pd
def load_titanic_data(filename, titanic_path=TITANIC_PATH):
 csv_path = os.path.join(titanic_path, filename)
 return pd.read_csv(csv_path)
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")
train_data.head()
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")
train_data.info()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
 ("imputer", SimpleImputer(strategy="median")),
 ("scaler", StandardScaler())
 ])
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
 ("imputer", SimpleImputer(strategy="most_frequent")),
 ("cat_encoder", OneHotEncoder(sparse=False)),
 ])
from sklearn.compose import ColumnTransformer
num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]
preprocess_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", cat_pipeline, cat_attribs),
 ])
X_train = preprocess_pipeline.fit_transform(
 train_data[num_attribs + cat_attribs])
X_train


In [ ]:
y_train = train_data["Survived"]
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100,
random_state=42)
forest_clf.fit(X_train, y_train)
RandomForestClassifier(random_state=42)
X_test = preprocess_pipeline.transform(test_data[num_attribs +
cat_attribs])
y_pred = forest_clf.predict(X_test)
from sklearn.model_selection import cross_val_score
forest_scores = cross_val_score(forest_clf, X_train, y_train,
cv=10)
forest_scores.mean()
0.8137578027465668
from sklearn.svm import SVC
svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM","Random
Forest"))
plt.ylabel("Accuracy", fontsize=14)
plt.show()

In [ ]:
import os
import tarfile
import urllib.request
DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")
def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL,
spam_path=SPAM_PATH):
 if not os.path.isdir(spam_path):
 os.makedirs(spam_path)
 for filename, url in (("ham.tar.bz2", ham_url),
("spam.tar.bz2", spam_url)):
 path = os.path.join(spam_path, filename)
 if not os.path.isfile(path):
 urllib.request.urlretrieve(url, path)
 tar_bz2_file = tarfile.open(path)
 tar_bz2_file.extractall(path=spam_path)
 tar_bz2_file.close()
fetch_spam_data()
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if
len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if
len(name) > 20]
len(ham_filenames)

In [ ]:
import email
import email.policy
def load_email(is_spam, filename, spam_path=SPAM_PATH):
 directory = "spam" if is_spam else "easy_ham"
 with open(os.path.join(spam_path, directory, filename), "rb")
as f:
 return
email.parser.BytesParser(policy=email.policy.default).parse(f)
ham_emails = [load_email(is_spam=False, filename=name) for name in
ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in
spam_filenames]
print(ham_emails[1].get_content().strip())
print(spam_emails[6].get_content().strip())
def get_email_structure(email):
 if isinstance(email, str):
 return email
 payload = email.get_payload()
 if isinstance(payload, list):
 return "multipart({})".format(", ".join([
 get_email_structure(sub_email)
 for sub_email in payload
 ]))
 else:
 return email.get_content_type()
from collections import Counter
def structures_counter(emails):
 structures = Counter()
 for email in emails:
 structure = get_email_structure(email)
 structures[structure] += 1
 return structures
structures_counter(ham_emails).most_common()
[('text/plain', 2408),
('multipart(text/plain, application/pgp-signature)', 66),
('multipart(text/plain, text/html)', 8),
('multipart(text/plain, text/plain)', 4),
('multipart(text/plain)', 3),
('multipart(text/plain, application/octet-stream)', 2),
('multipart(text/plain, text/enriched)', 1),
('multipart(text/plain, application/ms-tnef, text/plain)', 1),
('multipart(multipart(text/plain, text/plain, text/plain), applica
tion/pgp-signature)',
 1),
('multipart(text/plain, video/mng)', 1),
('multipart(text/plain, multipart(text/plain))', 1),
('multipart(text/plain, application/x-pkcs7-signature)', 1),
('multipart(text/plain, multipart(text/plain, text/plain), text/rf
c822-headers)',
 1),
('multipart(text/plain, multipart(text/plain, text/plain), multipa
rt(multipart(text/plain, application/x-pkcs7-signature)))',
 1),
('multipart(text/plain, application/x-java-applet)', 1)]
structures_counter(spam_emails).most_common()
[('text/plain', 218),
('text/html', 183),
('multipart(text/plain, text/html)', 45),
('multipart(text/html)', 20),
('multipart(text/plain)', 19),
('multipart(multipart(text/html))', 5),
('multipart(text/plain, image/jpeg)', 3),
('multipart(text/html, application/octet-stream)', 2),
('multipart(text/plain, application/octet-stream)', 1),
('multipart(text/html, text/plain)', 1),
('multipart(multipart(text/html), application/octet-stream, image/
jpeg)', 1),
('multipart(multipart(text/plain, text/html), image/gif)', 1),
('multipart/alternative', 1)]
for header, value in spam_emails[0].items():
 print(header,":",value)
spam_emails[0]["Subject"]
'Life Insurance - Why Pay More?'
import numpy as np
from sklearn.model_selection import train_test_split
X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=42)
import re
from html import unescape
def html_to_plain_text(html):
 text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M |
re.S | re.I)
 text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M |
re.S | re.I)
 text = re.sub('<.*?>', '', text, flags=re.M | re.S)
 text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
 return unescape(text)
html_spam_emails = [email for email in X_train[y_train==1]
 if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")
print(html_to_plain_text(sample_html_spam.get_content())[:1000],
"...")
def email_to_text(email):
 html = None
 for part in email.walk():
 ctype = part.get_content_type()
 if not ctype in ("text/plain", "text/html"):
 continue
 try:
 content = part.get_content()
 except: # in case of encoding issues
 content = str(part.get_payload())
 if ctype == "text/plain":
 return content
 else:
 html = content
 if html:
 return html_to_plain_text(html)
print(email_to_text(sample_html_spam)[:100], "...")
try:
 import nltk
 stemmer = nltk.PorterStemmer()
 for word in ("Computations", "Computation", "Computing",
"Computed", "Compute", "Compulsive"):
 print(word, "=>", stemmer.stem(word))
except ImportError:
 print("Error: stemming requires the NLTK module.")
 stemmer = None
if IS_COLAB or IS_KAGGLE:
 %pip install -q -U urlextract

In [ ]:
import urlextract # may require an Internet connection to
download root domain names

 url_extractor = urlextract.URLExtract()
 print(url_extractor.find_urls("Will it detect github.com and
https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
 print("Error: replacing URLs requires the urlextract module.")
 url_extractor = None
from sklearn.base import BaseEstimator, TransformerMixin
class EmailToWordCounterTransformer(BaseEstimator,
TransformerMixin):
 def __init__(self, strip_headers=True, lower_case=True,
remove_punctuation=True,
 replace_urls=True, replace_numbers=True,
stemming=True):
 self.strip_headers = strip_headers
 self.lower_case = lower_case
 self.remove_punctuation = remove_punctuation
 self.replace_urls = replace_urls
 self.replace_numbers = replace_numbers
 self.stemming = stemming
 def fit(self, X, y=None):
 return self
 def transform(self, X, y=None):
 X_transformed = []
 for email in X:
 text = email_to_text(email) or ""
 if self.lower_case:
 text = text.lower()
 if self.replace_urls and url_extractor is not None:
 urls = list(set(url_extractor.find_urls(text)))
 urls.sort(key=lambda url: len(url), reverse=True)
 for url in urls:
 text = text.replace(url, " URL ")
 if self.replace_numbers:
 text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?',
'NUMBER', text)
 if self.remove_punctuation:
 text = re.sub(r'\W+', ' ', text, flags=re.M)
 word_counts = Counter(text.split())
 if self.stemming and stemmer is not None:
 stemmed_word_counts = Counter()
 for word, count in word_counts.items():
 stemmed_word = stemmer.stem(word)
 stemmed_word_counts[stemmed_word] += count
 word_counts = stemmed_word_counts
 X_transformed.append(word_counts)
 return np.array(X_transformed)
X_few = X_train[:3]
X_few_wordcounts =
EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts


In [ ]:
from scipy.sparse import csr_matrix
class WordCounterToVectorTransformer(BaseEstimator,
TransformerMixin):
 def __init__(self, vocabulary_size=1000):
 self.vocabulary_size = vocabulary_size
 def fit(self, X, y=None):
 total_count = Counter()
 for word_count in X:
 for word, count in word_count.items():
 total_count[word] += min(count, 10)
 most_common =
total_count.most_common()[:self.vocabulary_size]
 self.vocabulary_ = {word: index + 1 for index, (word,
count) in enumerate(most_common)}
 return self
 def transform(self, X, y=None):
 rows = []
 cols = []
 data = []
 for row, word_count in enumerate(X):
 for word, count in word_count.items():
 rows.append(row)
 cols.append(self.vocabulary_.get(word, 0))
 data.append(count)
 return csr_matrix((data, (rows, cols)), shape=(len(X),
self.vocabulary_size + 1))
vocab_transformer =
WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors
<3x11 sparse matrix of type '<class 'numpy.longlong'>'
with 20 stored elements in Compressed Sparse Row format>
X_few_vectors.toarray()

In [ ]:
from sklearn.pipeline import Pipeline
preprocess_pipeline = Pipeline([
 ("email_to_wordcount", EmailToWordCounterTransformer()),
 ("wordcount_to_vector", WordCounterToVectorTransformer()),
])
X_train_transformed = preprocess_pipeline.fit_transform(X_train)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
log_clf = LogisticRegression(solver="lbfgs", max_iter=1000,
random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train,
cv=3, verbose=3)
score.mean()
0.9858333333333333
from sklearn.metrics import precision_score, recall_score
X_test_transformed = preprocess_pipeline.transform(X_test)
log_clf = LogisticRegression(solver="lbfgs", max_iter=1000,
random_state=42)
log_clf.fit(X_train_transformed, y_train)
y_pred = log_clf.predict(X_test_transformed)
print("Precision: {:.2f}%".format(100 * precision_score(y_test,
y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))